In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

# To make our relative library imports work
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
grandparent_dir = os.path.dirname(parent_dir)
sys.path.append(grandparent_dir)

In [3]:
import pickle

import pandas as pd
import altair as alt
import networkx as nx
import numpy as np
import numpy as np

from dataclasses import dataclass
from urllib.parse import unquote
from tqdm import tqdm

# Local Imports
from Library.embedding_extraction import extract_embeddings
from Library.narrative_landscape import NarrativeLandscape
from Library.storyline import Storyline
from metrics import similarity_metric, dtw_metric, absolute_metrics

/Users/faustogerman/miniforge3/envs/narrative-trails/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu


In [4]:
@dataclass
class Config:
    # The filename for the source data
    foldername = "wikispeedia"
    # The column with the contents of the article
    data_column = "plain_text"
    # The "date" column. This column will be renamed to "date" later.
    date_column = None
    # The column used as title for visualization and summary
    summary_column = "title"


CONFIG = Config()

In [5]:
data = pd.read_csv(f"../../data/{CONFIG.foldername}/text_data.csv")

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3928 entries, 0 to 3927
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   title       3928 non-null   object
 1   plain_text  3928 non-null   object
dtypes: object(2)
memory usage: 61.5+ KB


In [6]:
RAW_TEXT = data[CONFIG.data_column]

embeddings, _ = extract_embeddings(
    text=RAW_TEXT.tolist(),
    foldername=f"../../data/{CONFIG.foldername}",
    model_name="gpt4"  # OneOf[mpnet, gpt4]
)

File '../../data/wikispeedia/embed_data-gpt4.pickle' loaded successfully.


## Create Narrative Landscapes

In [7]:
links = pd.read_csv(
    "../../data/Wikispeedia/original/links.tsv",
    sep="\t",
    names=["src", "tgt"]
)

links["src"] = links["src"].map(lambda x: unquote(x))
links["tgt"] = links["tgt"].map(lambda x: unquote(x))

links = links[(links["src"].isin(data["title"])) & (links["tgt"].isin(data["title"]))]

ordered_data_titles = data["title"].tolist()

src_indices = links["src"].map(lambda x: ordered_data_titles.index(x))
tgt_indices = links["tgt"].map(lambda x: ordered_data_titles.index(x))

links_graph = nx.from_edgelist(list(zip(src_indices.tolist(), tgt_indices.tolist())))
links_constraints = nx.adjacency_matrix(links_graph, nodelist=range(len(ordered_data_titles))).todense()

### Regular Narrative Landscape

In [8]:
landscape = NarrativeLandscape(verbose=True)
landscape.fit(embeddings.numpy(), constraints=links_constraints)

Step 1/3: Constructing Projection Space with UMAP
Step 2/3: Discovering topics with HDBSCAN
	>>> Discovered 87 Topics
Step 3/3: Constructing Coherence Graph
	 >>> Computing base coherence
	 >>> Computing sparse coherence
		>>> Creating Undirected Graph
		>>> Finding Maximum Spanning Tree
		>>> Getting Min Weight
		----- BEFORE MST -----
		Critical Coherence: 0.5197929231231193
		Num Edges: 7712628
		Is Connected: True
		----- AFTER MST -----
		Num Edges: 3479213
		Is Connected: True
		----- AFTER Constraints -----
		Num Edges: 64117
		Is Connected: False
	 >>> Building NetworkX graph


### Narrative Landscape With Closeness Centrality Info

In [9]:
node_closeness = nx.closeness_centrality(links_graph)
node_closeness = np.array(list(node_closeness.values()))

landscape_closeness = NarrativeLandscape(verbose=True)
landscape_closeness.fit(embeddings.numpy(), constraints=links_constraints, node_ranks=node_closeness)

Step 1/3: Constructing Projection Space with UMAP
Step 2/3: Discovering topics with HDBSCAN
	>>> Discovered 87 Topics
Step 3/3: Constructing Coherence Graph
	 >>> Computing base coherence
	 >>> Computing sparse coherence
		>>> Creating Undirected Graph
		>>> Finding Maximum Spanning Tree
		>>> Getting Min Weight
		----- BEFORE MST -----
		Critical Coherence: 0.20110551007990723
		Num Edges: 7712628
		Is Connected: True
		----- AFTER MST -----
		Num Edges: 4259941
		Is Connected: True
		----- AFTER Constraints -----
		Num Edges: 68250
		Is Connected: False
	 >>> Building NetworkX graph


## Narrative Extraction


In [10]:
ground_truths = pd.read_csv(f"../../data/{CONFIG.foldername}/ground_truth_paths.csv")

# Remove paths with duplicated documents (within the same path)
ground_truths = ground_truths[ground_truths["path"].map(
    lambda x: not pd.Series(x.split(";")).duplicated().any()
)]

In [11]:
# This can take hours to finish, so we save the extracted narratives to a pickle file
try:
    with open('./extracted_narratives.pkl', 'rb') as handle:
        all_narratives = pickle.load(handle)
except FileNotFoundError:
    all_narratives = dict()
    for idx, true_storyline in tqdm(enumerate(ground_truths["path"]), total=len(ground_truths)):
        true_storyline = true_storyline.split(";")
        s = ordered_data_titles.index(unquote(true_storyline[0]))
        t = ordered_data_titles.index(unquote(true_storyline[-1]))

        if nx.has_path(landscape.nx_graph, s, t) and nx.has_path(landscape_closeness.nx_graph, s, t):
            # Shortest path
            sp = []
            shortest_simple_paths = nx.shortest_simple_paths(landscape.nx_graph, s, t)
            for _ in range(3):
                sp_path = next(shortest_simple_paths, None)
                if sp_path is not None:
                    sp.append(sp_path)

            # Regular narrative landscape
            nt = []
            paths_nodes = []
            for _ in range(3):
                trail, _ = landscape.extract_narrative(s, t, hidden_nodes=paths_nodes)
                if trail:
                    nt.append(trail)
                    paths_nodes.extend(trail[1:-1])

            # Narrative landscape with node closeness information
            nt_cc = []
            paths_nodes = []
            for _ in range(3):
                trail, _ = landscape_closeness.extract_narrative(s, t, hidden_nodes=paths_nodes)
                if trail:
                    nt_cc.append(trail)
                    paths_nodes.extend(trail[1:-1])

            all_narratives[(s, t, idx)] = {
                "random_points": [
                    [s, *np.random.randint(0, len(data), np.random.randint(5, 18)), t],
                    [s, *np.random.randint(0, len(data), np.random.randint(5, 18)), t],
                    [s, *np.random.randint(0, len(data), np.random.randint(5, 18)), t]
                ],
                "shortest_path": sp,
                "narrative_trails": nt,
                "narrative_trails_closeness": nt_cc,
            }
    with open('./extracted_narratives.pkl', 'wb') as handle:
        pickle.dump(all_narratives, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Evaluation

In [12]:
abs_comparison_counts = {"@1": 0, "@2": 0, "@3": 0}


def compute_absolute_metrics(kind):
    results = []  # (method, k, value)

    # Note that for metrics on the narratives with closeness centrality information,
    # we still use the original base coherence to keep all results in the same scale.
    for (_, _, story_idx), narratives in tqdm(all_narratives.items()):
        true_storyline = ground_truths["path"].iloc[story_idx].split(";")
        true_storyline = [ordered_data_titles.index(unquote(x)) for x in true_storyline]

        rp = narratives[f"random_points"]
        sp = narratives[f"shortest_path"]
        nt = narratives[f"narrative_trails"]
        cc = narratives[f"narrative_trails_closeness"]
        
        wikispeedia = absolute_metrics(true_storyline, landscape, kind)
        res_rp_k1 = absolute_metrics(rp[0], landscape, kind)
        res_sp_k1 = absolute_metrics(sp[0], landscape, kind)
        res_nt_k1 = absolute_metrics(nt[0], landscape, kind)
        res_rt_k1 = absolute_metrics(Storyline(landscape, nt[0]).reduce_redundancy(), landscape, kind)
        res_cc_k1 = absolute_metrics(cc[0], landscape, kind)
        res_rc_k1 = absolute_metrics(Storyline(landscape, cc[0]).reduce_redundancy(), landscape, kind)

        results.append(("Wikispeedia", 1, wikispeedia))
        results.append(("Random Points", 1, res_rp_k1))
        results.append(("Shortest Path", 1, res_sp_k1))
        results.append(("Narrative Trails", 1, res_nt_k1))
        results.append(("Reduced Trails", 1, res_rt_k1))
        results.append(("Narrative Trails (CC)", 1, res_cc_k1))
        results.append(("Reduced Trails (CC)", 1, res_rc_k1))

        abs_comparison_counts["@1"] += 1

        if len(rp) >= 2 and len(sp) >= 2 and len(nt) >= 2 and len(cc) >= 2:
            res_rp_k2 = absolute_metrics(rp[1], landscape, kind)
            res_sp_k2 = absolute_metrics(sp[1], landscape, kind)
            res_nt_k2 = absolute_metrics(nt[1], landscape, kind)
            res_rt_k2 = absolute_metrics(Storyline(landscape, nt[1]).reduce_redundancy(), landscape, kind)
            res_cc_k2 = absolute_metrics(cc[1], landscape, kind)
            res_rc_k2 = absolute_metrics(Storyline(landscape, cc[1]).reduce_redundancy(), landscape, kind)

            results.append(("Random Points", 2, sum((res_rp_k1, res_rp_k2)) / 2))
            results.append(("Shortest Path", 2, sum((res_sp_k1, res_sp_k2)) / 2))
            results.append(("Narrative Trails", 2, sum((res_nt_k1, res_nt_k2)) / 2))
            results.append(("Reduced Trails", 2, sum((res_rt_k1, res_rt_k2)) / 2))
            results.append(("Narrative Trails (CC)", 2, sum((res_cc_k1, res_cc_k2)) / 2))
            results.append(("Reduced Trails (CC)", 2, sum((res_rc_k1, res_rc_k2)) / 2))

            abs_comparison_counts["@2"] += 1

        if len(rp) == 3 and len(sp) == 3 and len(nt) == 3 and len(cc) == 3:
            res_rp_k3 = absolute_metrics(rp[2], landscape, kind)
            res_sp_k3 = absolute_metrics(sp[2], landscape, kind)
            res_nt_k3 = absolute_metrics(nt[2], landscape, kind)
            res_rt_k3 = absolute_metrics(Storyline(landscape, nt[2]).reduce_redundancy(), landscape, kind)
            res_cc_k3 = absolute_metrics(cc[2], landscape, kind)
            res_rc_k3 = absolute_metrics(Storyline(landscape, cc[2]).reduce_redundancy(), landscape, kind)

            results.append(("Random Points", 3, sum((res_rp_k1, res_rp_k2, res_rp_k3)) / 3))
            results.append(("Shortest Path", 3, sum((res_sp_k1, res_sp_k2, res_sp_k3)) / 3))
            results.append(("Narrative Trails", 3, sum((res_nt_k1, res_nt_k2, res_nt_k3)) / 3))
            results.append(("Reduced Trails", 3, sum((res_rt_k1, res_rt_k2, res_rt_k3)) / 3))
            results.append(("Narrative Trails (CC)", 3, sum((res_cc_k1, res_cc_k2, res_cc_k3)) / 3))
            results.append(("Reduced Trails (CC)", 3, sum((res_rc_k1, res_rc_k2, res_rc_k3)) / 3))

            abs_comparison_counts["@3"] += 1

    return results

In [13]:
gt_comparison_counts = {"@1": 0, "@2": 0, "@3": 0}


def compute_metric_with_ground_truth(metric, low_emb):
    results = []  # (method, k, value)

    for (_, _, story_idx), narratives in tqdm(all_narratives.items()):
        true_storyline = ground_truths["path"].iloc[story_idx].split(";")
        true_storyline = [ordered_data_titles.index(unquote(x)) for x in true_storyline]

        rp = narratives[f"random_points"]
        sp = narratives[f"shortest_path"]
        nt = narratives[f"narrative_trails"]
        cc = narratives[f"narrative_trails_closeness"]

        res_rp_k1 = metric(true_storyline, rp[0], low_emb)
        res_sp_k1 = metric(true_storyline, sp[0], low_emb)
        res_nt_k1 = metric(true_storyline, nt[0], low_emb)
        nt_st_line = Storyline(landscape, nt[0])
        res_rt_k1 = metric(true_storyline, nt_st_line.reduce_redundancy(), low_emb)
        res_cc_k1 = metric(true_storyline, cc[0], low_emb)
        nt_st_line = Storyline(landscape_closeness, cc[0])
        res_rc_k1 = metric(true_storyline, nt_st_line.reduce_redundancy(), low_emb)

        results.append(("Random Points", 1, res_rp_k1))
        results.append(("Shortest Path", 1, res_sp_k1))
        results.append(("Narrative Trails", 1, res_nt_k1))
        results.append(("Reduced Trails", 1, res_rt_k1))
        results.append(("Narrative Trails (CC)", 1, res_cc_k1))
        results.append(("Reduced Trails (CC)", 1, res_rc_k1))

        gt_comparison_counts["@1"] += 1

        if len(rp) >= 2 and len(sp) >= 2 and len(nt) >= 2 and len(cc) >= 2:
            res_rp_k2 = metric(true_storyline, rp[1], low_emb)
            res_sp_k2 = metric(true_storyline, sp[1], low_emb)
            res_nt_k2 = metric(true_storyline, nt[1], low_emb)
            nt_st_line = Storyline(landscape, nt[1])
            res_rt_k2 = metric(true_storyline, nt_st_line.reduce_redundancy(), low_emb)
            res_cc_k2 = metric(true_storyline, cc[1], low_emb)
            rc_st_line = Storyline(landscape_closeness, cc[1])
            res_rc_k2 = metric(true_storyline, rc_st_line.reduce_redundancy(), low_emb)

            results.append(("Random Points", 2, sum((res_rp_k1, res_rp_k2)) / 2))
            results.append(("Shortest Path", 2, sum((res_sp_k1, res_sp_k2)) / 2))
            results.append(("Narrative Trails", 2, sum((res_nt_k1, res_nt_k2)) / 2))
            results.append(("Reduced Trails", 2, sum((res_rt_k1, res_rt_k2)) / 2))
            results.append(("Narrative Trails (CC)", 2, sum((res_cc_k1, res_cc_k2)) / 2))
            results.append(("Reduced Trails (CC)", 2, sum((res_rc_k1, res_rc_k2)) / 2))

            gt_comparison_counts["@2"] += 1

        if len(rp) == 3 and len(sp) == 3 and len(nt) == 3 and len(cc) == 3:
            res_rp_k3 = metric(true_storyline, rp[2], low_emb)
            res_sp_k3 = metric(true_storyline, sp[2], low_emb)
            res_nt_k3 = metric(true_storyline, nt[2], low_emb)
            nt_st_line = Storyline(landscape, nt[2])
            res_rt_k3 = metric(true_storyline, nt_st_line.reduce_redundancy(), low_emb)
            res_cc_k3 = metric(true_storyline, cc[2], low_emb)
            rc_st_line = Storyline(landscape_closeness, cc[2])
            res_rc_k3 = metric(true_storyline, rc_st_line.reduce_redundancy(), low_emb)

            results.append(("Random Points", 3, sum((res_rp_k1, res_rp_k2, res_rp_k3)) / 3))
            results.append(("Shortest Path", 3, sum((res_sp_k1, res_sp_k2, res_sp_k3)) / 3))
            results.append(("Narrative Trails", 3, sum((res_nt_k1, res_nt_k2, res_nt_k3)) / 3))
            results.append(("Reduced Trails", 3, sum((res_rt_k1, res_rt_k2, res_rt_k3)) / 3))
            results.append(("Narrative Trails (CC)", 3, sum((res_cc_k1, res_cc_k2, res_cc_k3)) / 3))
            results.append(("Reduced Trails (CC)", 3, sum((res_rc_k1, res_rc_k2, res_rc_k3)) / 3))

            gt_comparison_counts["@3"] += 1

    return results

In [14]:
df_rel = pd.DataFrame(compute_absolute_metrics("reliability"), columns=["Method", "k", "Rel"])
df_rel = df_rel.groupby(by=["Method", "k"]).mean()

df_coh = pd.DataFrame(compute_absolute_metrics("min_coherence"), columns=["Method", "k", "Coh"])
df_coh = df_coh.groupby(by=["Method", "k"]).mean()

gt_sim_res = compute_metric_with_ground_truth(similarity_metric, landscape.low_dim_embeds)
df_sim = pd.DataFrame(gt_sim_res, columns=["Method", "k", "Sim"]).groupby(by=["Method", "k"]).mean()

gt_dist_res = compute_metric_with_ground_truth(dtw_metric, landscape.low_dim_embeds)
df_dtw = pd.DataFrame(gt_dist_res, columns=["Method", "k", "DTW"]).groupby(by=["Method", "k"]).mean()

100%|██████████| 10607/10607 [00:24<00:00, 424.50it/s]


In [15]:
print(len(all_narratives.items()), "of", len(ground_truths))
print(gt_comparison_counts) # Divide by 2
print(abs_comparison_counts) # Divide by 2

10607 of 10832
{'@1': 21214, '@2': 19172, '@3': 16770}
{'@1': 21214, '@2': 19172, '@3': 16770}


In [16]:
# Reset the index
df = pd.concat([df_rel, df_coh, df_dtw, df_sim], axis=1)
df = df.reset_index().round(3)

# Set the new index
df = df.set_index(['Method'])

# Melt the DataFrame
df_melted = df.reset_index().melt(
    id_vars=['Method', 'k'],
    value_vars=['Rel', 'Coh', 'Sim', 'DTW'],
    var_name='metric',
    value_name='value'
)

# Pivot the DataFrame
df_pivot = df_melted.pivot_table(
    index=['Method'],
    columns=['metric', 'k'],
    values='value'
)

# Flatten the MultiIndex columns
df_pivot.columns = [
    f"{metric}_{k}" for metric, k in df_pivot.columns
]

df_pivot = df_pivot[["Coh_1", "Coh_2", "Coh_3", "Rel_1", "Rel_2", "Rel_3", "Sim_1", "Sim_2", "Sim_3", "DTW_1", "DTW_2", "DTW_3"]]
df_pivot = df_pivot.loc[[
    "Wikispeedia",
    "Random Points",
    "Shortest Path",
    "Narrative Trails",
    "Reduced Trails",
    "Narrative Trails (CC)",
    "Reduced Trails (CC)",
]]

df_pivot.to_csv("benchmark_results.csv")
df_pivot

,Coh_1,Coh_2,Coh_3,Rel_1,Rel_2,Rel_3,Sim_1,Sim_2,Sim_3,DTW_1,DTW_2,DTW_3
Method,,,,,,,,,,,,
Wikispeedia,0.419,NaN,NaN,0.609,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Random Points,0.320,0.321,0.322,0.454,0.455,0.456,0.347,0.347,0.347,2.200,2.201,2.200
Shortest Path,0.558,0.560,0.563,0.614,0.615,0.620,0.742,0.742,0.746,0.967,0.978,0.971
Narrative Trails,0.709,0.704,0.704,0.776,0.769,0.767,0.788,0.785,0.787,1.029,1.049,1.063
Reduced Trails,0.668,0.667,0.669,0.760,0.756,0.754,0.768,0.768,0.771,1.056,1.077,1.088
Narrative Trails (CC),0.640,0.631,0.630,0.753,0.748,0.746,0.777,0.778,0.766,1.029,1.049,1.093
Reduced Trails (CC),0.630,0.625,0.624,0.736,0.734,0.734,0.758,0.760,0.750,1.069,1.082,1.119


## Example

In [17]:
story_idx = pd.Series(all_narratives.keys()).sample(1, random_state=0).values[0]
gt = [ordered_data_titles.index(unquote(x)) for x in ground_truths["path"].iloc[story_idx[-1]].split(";")]

print("Reliability:", Storyline(landscape, gt).reliability())
print("Coherences:", Storyline(landscape, gt).path_base_coherence())

data.iloc[gt]["title"]

Reliability: 0.6141220377905909
Coherences: [0.67154048 0.66929064 0.63787183 0.58873993 0.55455038 0.57311691]


913     Cornell_University
3668         United_States
926                Country
1101                 Earth
3254         South_America
286              Argentina
624           Buenos_Aires
Name: title, dtype: object

In [18]:
sp = all_narratives[story_idx]["shortest_path"][0]
print("Reliability:", Storyline(landscape, sp).reliability())
print("Coherences:", Storyline(landscape, sp).path_base_coherence())

data.iloc[sp]["title"]

Reliability: 0.666284836733653
Coherences: [0.67469644 0.65797811]


913    Cornell_University
464               Beijing
624          Buenos_Aires
Name: title, dtype: object

In [19]:
nt = all_narratives[story_idx]["narrative_trails"][0]

print("Reliability:", Storyline(landscape, nt).reliability())
print("Coherences:", Storyline(landscape, nt).path_base_coherence())

data.iloc[nt]["title"]

Reliability: 0.7292108849616922
Coherences: [0.70087677 0.72335243 0.72531052 0.76853098 0.71967434 0.7322392
 0.71036678 0.7557472 ]


913       Cornell_University
665                   Canada
17           1973_oil_crisis
268    Arab-Israeli_conflict
269              Arab_League
116            African_Union
86                     Abuja
586                   Brazil
624             Buenos_Aires
Name: title, dtype: object

In [20]:
nt_reduced = Storyline(landscape, nt).reduce_redundancy()

print("Reliability:", Storyline(landscape, nt_reduced).reliability())
print("Coherences:", Storyline(landscape, nt_reduced).path_base_coherence())

data.iloc[nt_reduced]["title"]

Reliability: 0.7292108849616922
Coherences: [0.70087677 0.72335243 0.72531052 0.76853098 0.71967434 0.7322392
 0.71036678 0.7557472 ]


913       Cornell_University
665                   Canada
17           1973_oil_crisis
268    Arab-Israeli_conflict
269              Arab_League
116            African_Union
86                     Abuja
586                   Brazil
624             Buenos_Aires
Name: title, dtype: object

In [21]:
# Create DataFrame
df = pd.DataFrame(data.iloc[nt_reduced]["title"])
df['index'] = range(len(df))

# Base chart: Line
line = alt.Chart(df).mark_line(color='blue').encode(
    x=alt.X('index', axis=None),
    y=alt.value(0)
)

# Points
points = alt.Chart(df).mark_point(color='blue').encode(
    x='index',
    y=alt.value(0)
)

# Text labels
text = alt.Chart(df).mark_text(
    align='left',
    baseline='bottom',
    dx=2,
    dy=-5,
    angle=315
).encode(
    x='index',
    y=alt.value(0),
    text='title'
)

# Combine charts
chart = (line + points + text).configure_view(
    strokeWidth=0
).properties(
    width=300,
    height=50
)

# Display the chart
chart

alt.LayerChart(...)

## Statistical Significance Analysis

In [22]:
df_coh_full = pd.DataFrame(compute_absolute_metrics("min_coherence"), columns=["Method", "k", "Coh"])

df_rel_full = pd.DataFrame(compute_absolute_metrics("reliability"), columns=["Method", "k", "Rel"])

df_sim_full = compute_metric_with_ground_truth(similarity_metric, landscape.low_dim_embeds)
df_sim_full = pd.DataFrame(df_sim_full, columns=["Method", "k", "Sim"])

gt_dist_full = compute_metric_with_ground_truth(dtw_metric, landscape.low_dim_embeds)
gt_dist_full = pd.DataFrame(gt_dist_full, columns=["Method", "k", "DTW"])

100%|██████████| 10607/10607 [00:22<00:00, 463.52it/s]


### Test for Normality
The p-values suggest our results are not normally distributed

In [23]:
import scipy.stats as stats

for k in [1, 2, 3]:
    for method in ["Wikispeedia", "Random Points", "Shortest Path", "Narrative Trails", "Narrative Trails (CC)", "Reduced Trails", "Reduced Trails (CC)"]:
        coh = df_coh_full[(df_coh_full["Method"] == method) & (df_coh_full["k"] == k)]["Coh"]
        rel = df_rel_full[(df_rel_full["Method"] == method) & (df_rel_full["k"] == k)]["Rel"]
        dtw = gt_dist_full[(gt_dist_full["Method"] == method) & (gt_dist_full["k"] == k)]["DTW"]

        print(method, k)
        if method != "Wikispeedia" or (method == "Wikispeedia" and k == 1):
            print("Coh:", stats.normaltest(coh))
            print("Rel:", stats.normaltest(rel))

        if method != "Wikispeedia":
            sim = df_sim_full[(df_sim_full["Method"] == method) & (df_sim_full["k"] == k)]["Sim"]
            print("Sim:", stats.normaltest(sim))
            print("DTW:", stats.normaltest(dtw))

        print()

Wikispeedia 1
Coh: NormaltestResult(statistic=3128.683552210846, pvalue=0.0)
Rel: NormaltestResult(statistic=211.07674469604075, pvalue=1.4630772596578454e-46)

Random Points 1
Coh: NormaltestResult(statistic=6073.777290310638, pvalue=0.0)
Rel: NormaltestResult(statistic=2849.040507312121, pvalue=0.0)
Sim: NormaltestResult(statistic=171.86024384709032, pvalue=4.79757993339112e-38)
DTW: NormaltestResult(statistic=9.378519921045301, pvalue=0.009193487182870018)

Shortest Path 1
Coh: NormaltestResult(statistic=3128.455683142572, pvalue=0.0)
Rel: NormaltestResult(statistic=536.7038359551455, pvalue=2.8591886698922428e-117)
Sim: NormaltestResult(statistic=3048.926615269689, pvalue=0.0)
DTW: NormaltestResult(statistic=1432.4581670077437, pvalue=8.82391377511e-312)

Narrative Trails 1
Coh: NormaltestResult(statistic=1283.5000961884211, pvalue=1.956569825717572e-279)
Rel: NormaltestResult(statistic=2963.590800308403, pvalue=0.0)
Sim: NormaltestResult(statistic=3514.702690178826, pvalue=0.0)
DT

### Statistical Significance
In most cases, the results from our Narrative Trails algorithm are statistically significant. However, a few instances between Reduced Trails and Shortest Path show that the results are not statically significant. This suggests that our redundancy reduction algorithm makes the maximum capacity path approximate the shortest simple path between the source and target documents.

In [24]:
for k in [1, 2, 3]:
    for main_method in ["Narrative Trails", "Narrative Trails (CC)", "Reduced Trails", "Reduced Trails (CC)"]:
        main_method_coh = df_coh_full[(df_coh_full["Method"] == main_method) & (df_coh_full["k"] == k)]["Coh"]
        main_method_rel = df_rel_full[(df_rel_full["Method"] == main_method) & (df_rel_full["k"] == k)]["Rel"]
        main_method_dtw = gt_dist_full[(gt_dist_full["Method"] == main_method) & (gt_dist_full["k"] == k)]["DTW"]
        main_method_sim = df_sim_full[(df_sim_full["Method"] == main_method) & (df_sim_full["k"] == k)]["Sim"]

        for secondary_method in ["Wikispeedia", "Random Points", "Shortest Path"]:
            if secondary_method == "Wikispeedia" and k != 1:
                continue
            print(f"{main_method} vs {secondary_method} @ {k}")

            secondary_method_coh = df_coh_full[(df_coh_full["Method"] == secondary_method) & (df_coh_full["k"] == k)]["Coh"]
            secondary_method_rel = df_rel_full[(df_rel_full["Method"] == secondary_method) & (df_rel_full["k"] == k)]["Rel"]
            secondary_method_dtw = gt_dist_full[(gt_dist_full["Method"] == secondary_method) & (gt_dist_full["k"] == k)]["DTW"]
            secondary_method_sim = df_sim_full[(df_sim_full["Method"] == secondary_method) & (df_sim_full["k"] == k)]["Sim"]

            print("Coh:", stats.mannwhitneyu(main_method_coh, secondary_method_coh, alternative="two-sided"))
            print("Rel:", stats.mannwhitneyu(main_method_rel, secondary_method_rel, alternative="two-sided"))

            if secondary_method != "Wikispeedia":
                print("Sim:", stats.mannwhitneyu(main_method_sim, secondary_method_sim, alternative="two-sided"))
                print("DTW:", stats.mannwhitneyu(main_method_dtw, secondary_method_dtw, alternative="two-sided"))

            print()
        print("-" * 50)

Narrative Trails vs Wikispeedia @ 1
Coh: MannwhitneyuResult(statistic=109901629.5, pvalue=0.0)
Rel: MannwhitneyuResult(statistic=108696525.0, pvalue=0.0)

Narrative Trails vs Random Points @ 1
Coh: MannwhitneyuResult(statistic=112480688.0, pvalue=0.0)
Rel: MannwhitneyuResult(statistic=112505148.0, pvalue=0.0)
Sim: MannwhitneyuResult(statistic=108778843.0, pvalue=0.0)
DTW: MannwhitneyuResult(statistic=3136681.0, pvalue=0.0)

Narrative Trails vs Shortest Path @ 1
Coh: MannwhitneyuResult(statistic=106758381.5, pvalue=0.0)
Rel: MannwhitneyuResult(statistic=110695181.5, pvalue=0.0)
Sim: MannwhitneyuResult(statistic=59672655.5, pvalue=1.7904938171531334e-14)
DTW: MannwhitneyuResult(statistic=63480345.5, pvalue=4.84266614896973e-59)

--------------------------------------------------
Narrative Trails (CC) vs Wikispeedia @ 1
Coh: MannwhitneyuResult(statistic=106077396.5, pvalue=0.0)
Rel: MannwhitneyuResult(statistic=106863231.0, pvalue=0.0)

Narrative Trails (CC) vs Random Points @ 1
Coh: Mann